In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as T
import wandb
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader
from tqdm import tqdm

from nexar.data import NexarDataModule, NexarDataset, pad_to_square
from nexar.model import NexarClassifier


In [ ]:
# Set seed for reproducibility
SEED = None

### Train

In [ ]:
random_seed = SEED or np.random.randint(0, 1e6)
seed_everything(random_seed, workers=True)

# Initialize trainin data module
train_df = pd.read_parquet("../data/processed/train.parquet")
datamodule = NexarDataModule(
    train_df=train_df,
    batch_size=32,
    val_size=0.1,
    transform=T.Compose([
        T.Lambda(pad_to_square),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        T.RandomHorizontalFlip(),
        T.RandomAffine(degrees=2, translate=(0.05, 0.05), scale=(0.95, 1.05), shear=2),
    ]),
    test_transform=T.Compose([
        T.Lambda(pad_to_square),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
)

# Initialize model
model = NexarClassifier(lr=1e-3)

# Initialize trainer
trainer = Trainer(
    max_epochs=20,
    logger=WandbLogger(project="nexar-collision-prediction", save_dir="../logs"),
    callbacks=[
        ModelCheckpoint(monitor="val_acc", mode="max", save_top_k=1, filename="{epoch:02d}-{val_acc:.2f}"),
        EarlyStopping(monitor="val_acc", mode="max", patience=8, verbose=True),
    ],
    deterministic=True,
)

# Log seed
trainer.logger.experiment.config.update({"seed": random_seed})

# Train the model
trainer.fit(model, datamodule=datamodule)
wandb.finish()


### Predict

In [17]:
device = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: mps


In [ ]:
# Load the best model
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = NexarClassifier.load_from_checkpoint(best_model_path)
best_model.to(device)
best_model.eval()
best_model_id = best_model_path.split("/")[3]

print(f"Best model path: {best_model_path}")
print(f"Best model id: {best_model_id}")


Best model path: ../logs/nexar-collision-prediction/39exxit9/checkpoints/epoch=17-val_acc=0.79.ckpt
Best model id: 39exxit9


In [19]:
test_df = pd.read_parquet("../data/processed/test.parquet")

predictions = {}
indices = [0, 1, 2]
weights = [0.2, 0.3, 0.5]

# Get predictions for each frame
for frame_idx in indices:
    test_dataset = NexarDataset(
        test_df, 
        frame_idx=frame_idx, 
        return_label=False, 
        transform=datamodule.test_transform,
    )
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=False)
    
    preds = []
    for batch in tqdm(test_dataloader):
        batch = {k: v.to(best_model.device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        with torch.no_grad():
            pred = best_model(batch)
        pred = torch.sigmoid(pred).squeeze().detach().tolist()
        preds.extend(pred)
    
    predictions[frame_idx] = preds

# Take weighted average of predictions
final_predictions = np.zeros(len(test_df))
for i, frame_idx in enumerate(indices):
    final_predictions += np.array(predictions[frame_idx]) * weights[i]
final_predictions = final_predictions / sum(weights)

# Save predictions
submission_df = pd.DataFrame({"id": test_df["id"].apply(lambda x: str(x).zfill(5)), "target": final_predictions})
submission_df.to_csv(f"../data/processed/submission_{best_model_id}.csv", index=False)
submission_df.head()


100%|██████████| 21/21 [01:21<00:00,  3.90s/it]


,id,target
0,00204,0.253067
1,00030,0.886701
2,00146,0.917146
3,00020,0.226513
4,00511,0.991201
